# Treinamento dos modelos

In [1]:
import pandas as pd
import sklearn

In [2]:
df = pd.read_csv('cancellation_prediction.csv')
df = df.loc[:, df.columns != 'id_person_booking']
df.dropna(how='any', inplace=True)
df.describe()

,cancellation,days_between_booking_arrival,year_arrival_date,week_number_arrival_date,day_of_month_arrival_date,num_weekend_nights,num_workweek_nights,num_adults,num_children,num_babies,...,distribution_channel,repeated_guest,num_previous_cancellations,num_previous_stays,changes_between_booking_arrival,id_travel_agency_booking,customer_type,avg_price,required_car_parking_spaces,total_of_special_requests
count,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,...,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000,102894.000000
mean,0.390314,111.740092,2016.156977,27.339155,15.786771,0.978181,2.589655,1.899926,0.111231,0.007697,...,1.856658,0.014034,0.077215,0.024365,0.201780,86.545532,0.453729,104.561859,0.053035,0.605458
std,0.487823,107.681013,0.706117,13.279990,8.794042,1.003991,1.905941,0.490891,0.411982,0.097089,...,0.519669,0.117631,0.758185,0.435759,0.624954,110.714259,0.822130,46.895098,0.225359,0.799342
min,0.000000,0.000000,2015.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,26.000000,2016.000000,17.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,73.000000,0.000000,0.000000
50%,0.000000,79.000000,2016.000000,28.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,14.000000,0.000000,96.300000,0.000000,0.000000
75%,1.000000,169.000000,2017.000000,38.000000,24.000000,2.000000,3.000000,2.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,0.000000,0.000000,229.000000,0.000000,127.800000,0.000000,1.000000
max,1.000000,629.000000,2017.000000,53.000000,31.000000,16.000000,41.000000,26.000000,10.000000,10.000000,...,4.000000,1.000000,26.000000,50.000000,18.000000,535.000000,3.000000,300.000000,3.000000,5.000000


In [3]:
month_number = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

In [4]:
df['month'] = df['month_arrival_date'].apply(lambda x: month_number[x])

In [5]:
no_change_columns = [
    'days_between_booking_arrival', 'market_segment', 'distribution_channel', 'num_previous_cancellations',
    'total_of_special_requests', 'required_car_parking_spaces', 'changes_between_booking_arrival', 'repeated_guest',
    'year_arrival_date', 'month'
]

In [6]:
dummies_columns = [
    'type'
]

In [7]:
X = pd.concat((pd.DataFrame(), *[ df[x] for x in no_change_columns ], *[ pd.get_dummies(df[x]) for x in dummies_columns ]), axis=1)
Y = df.loc[:, 'cancellation']

## Cross-validation não estratificada com todos os dados

In [8]:
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
import numpy as np

- Os modelos serão testados utilizando Cross-validation e Shuffle Split
- Os dados de treinamento serão embaralhados antes de serem divididos em 10 partições, treinados em 9 e testado em 1
- Será utilizada a métrica F1 para avaliar os modelos

In [9]:
results = cross_validate(GaussianNB(), X, Y, scoring=['f1'], cv=ShuffleSplit(random_state=33))
np.mean(results['test_f1'])

0.5977106639901562

In [10]:
results = cross_validate(DecisionTreeClassifier(), X, Y, scoring=['f1'], cv=ShuffleSplit(random_state=33))
np.mean(results['test_f1'])

0.7449795957303812

In [12]:
results = cross_validate(AdaBoostClassifier(), X, Y, scoring=['f1'], cv=ShuffleSplit(random_state=33))
np.mean(results['test_f1'])

0.6477055976417729

In [13]:
results = cross_validate(ExtraTreesClassifier(), X, Y, scoring=['f1'], cv=ShuffleSplit(random_state=33))
np.mean(results['test_f1'])

0.7514774137738974

- O melhor resultado foi apresentado com `ExtraTreesClassifier` com 0.75 de F1, próximo do `DecisionTreeClassifier` com 0.75. Sem qualquer tunning de hiper-parâmentros, os algorítmos de árvores parecem convergir com dados apresentados
- Uma busca nos hiper-parâmentros ou mudaça nos dados podem melhorar os resultados
- O NaiveBayes não apresentou resultado bons, provavelmente devido a baixa correlação entre as features
- Uma árvore de decisão consegue selecionar as features que vão melhor separar os dados nas classes dadas, o que pode ser melhor para esse cenário

## Treino separado por ano

- Treinar com os dados de 2015 e testar com 2016, 2017
- Treinar com os dados de 2015,2016 e testar com 2017

In [14]:
from sklearn.metrics import f1_score

In [15]:
idx_15 = X.query('year_arrival_date == 2015').index
idx_16 = X.query('year_arrival_date == 2016').index
idx_17 = X.query('year_arrival_date == 2017').index

In [16]:
train_x15 = X.loc[idx_15]
train_y15 = Y.loc[idx_15]

test_x1617 = X.loc[idx_16.append(idx_17)]
test_y1617 = Y.loc[idx_16.append(idx_17)]

In [17]:
dtc = DecisionTreeClassifier()
dtc.fit(train_x15, train_y15)
yy = dtc.predict(test_x1617)
f1_score(test_y1617, yy)

0.3587420636394014

In [18]:
train_x1516 = X.loc[idx_15.append(idx_16)]
train_y1516 = Y.loc[idx_15.append(idx_16)]

test_x17 = X.loc[idx_17]
test_y17 = Y.loc[idx_17]

In [19]:
dtc = DecisionTreeClassifier()
dtc.fit(train_x1516, train_y1516)
yy = dtc.predict(test_x17)
f1_score(test_y17, yy)

0.4832091114647014

- Como os dados não parecem ter uma grande variação de ano a ano, limitar a quantide de dados para o treinamento vai limitar a qualidade do modelo
- Um treinamento separado por tempo seria mais indicado para séries temporais em que os valores anteriores têm forte correlação com os valores futuros
- No caso desses dados, um treinameto com todos os dados se mostrou mais vantajoso

## Conclusão

- Os dados não apresentam correlação forte entre as features, o que pode indicar que um modelo baseado em probabilidade (tipo Naive Bayes) não apresentaria bons resultados
- Um árvore de decisão, apesar de simples conseguiu resultados bons, 0.75 de F1
- Os resultados poderiam ser melhorados com fine tunning dos hiper-parâmetros ou o uso de ensemble de árvores (como o XGBoost)
- Com base na exploração dos dados, as seguintes hipóteses foram formuladas:
  - O tempo entre a data que foi feita a reserva e a data da chegada é correlacionado com o cacelamento
    - uma maior chance do cliente se arrepender e cancelar
  - O cliente fazer pedidos especiais diminui a chance de cancelamento
    - se um cliente estiver mais engajado existe menos chance de cancelar
  - Os cancelamentos são menores durante o fim/começo do ano
    - festas de fim de ano
  - A taxa de cancelamento parece ser consistente ano a ano